In [ ]:
import pandas as pd
from scipy import Model, odr
from uncertainties import ufloat


def polynom_second_grade(p, x):
    a, b, c = p
    return a + b * x + c * x**2


class RealUncertainData(odr.RealData):
    def __init__(self, x, y):
        nx = x
        ny = y
        sx = sy = None

        try:
            nx = [p.nominal_value for p in x]
            sx = [p.std_dev for p in x]
        except AttributeError:
            pass
        try:
            ny = [p.nominal_value for p in y]
            sy = [p.std_dev for p in y]
        except AttributeError:
            pass
        super().__init__(x=nx, y=ny, sx=sx, sy=sy)

In [ ]:
df = pd.DataFrame()

model = Model(polynom_second_grade)
real_data = RealUncertainData(
    x=df.index,
    y=df["foobar"].to_numpy(),
)

odr = odr.ODR(real_data, model, beta0=[1.0] * 3)

odr_fit = odr.run()
df["fit"] = model.fcn(odr_fit.beta, df.index)